Alexnet

In [2]:
import torch
from torchvision import models, transforms, utils
from torch.utils.data import Dataset, DataLoader
from torch.utils.tensorboard import SummaryWriter
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import pickle
import matplotlib.pyplot as plt
%matplotlib inline
import math
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score

In [ ]:
X_train = pickle.load(open(f'X_train298.pkl', 'rb'))
y_train = pickle.load(open(f'y_train298.pkl', 'rb'))
X_test = pickle.load(open(f'X_test298.pkl', 'rb'))
y_test = pickle.load(open(f'y_test298.pkl', 'rb'))

In [3]:
X_train = pickle.load(open(f'X2669.pkl', 'rb'))
y_train = pickle.load(open(f'y2669.pkl', 'rb'))

In [4]:
criterion = nn.CrossEntropyLoss()

In [5]:
def getNormalized(X,s):
    flattened_channels = X.reshape(3,-1)
    channel_mean = flattened_channels.mean(dim = 1)
    channel_stddev = flattened_channels.std(dim = 1)
    preprocess2 = transforms.Compose([
                      transforms.Normalize(channel_mean, channel_stddev)
    ])


    temptwo = torch.tensor([])
    for i in range(X.shape[0]):
        a = preprocess2(X[i])
        temptwo = torch.cat([temptwo, a.reshape(1,3,s,s)])
  
    return temptwo


def imageSetResize(newSize,X):
    preprocess1 = transforms.Compose([
                        transforms.ToPILImage(),
                        transforms.Resize(newSize),
                        transforms.ToTensor()])
  
    temp = torch.tensor([])
    for i in range(X.shape[0]):
        a = preprocess1(X[i])
        temp = torch.cat([temp, a.reshape(1,3,newSize,newSize)])

    return temp 


def splitTrainTest(X,y):
    shuffled_indices = torch.randperm(X.shape[0])
    ul = math.floor(0.8*X.shape[0])
    train_indices = shuffled_indices[:ul]
    test_indices = shuffled_indices[ul:]
    # train_indices.shape[0] + test_indices.shape[0]
    X_train = X[train_indices]
    y_train = y[train_indices]  
    X_test = X[test_indices]
    y_test = y[test_indices]
    print('y_train -> [0]:{} [1]:{}'.format((y_train == 0).sum().item(), (y_train == 1).sum().item()))
    print('y_test -> [0]:{} [1]:{}'.format((y_test == 0).sum().item(), (y_test == 1).sum().item()))
    return X_train, y_train, X_test, y_test


def labelize(p):
    labelized_preds = []
    for i in p:
        l = 0. if i[0]>i[1] else 1.
        labelized_preds.append(l)

    return torch.tensor(labelized_preds)



def shuffle_and_batch(X,y,num,bs):
    shuffled_indices = torch.randperm(X.shape[0])
    newX = X[shuffled_indices]
    newY = y[shuffled_indices]

    X_batches = []
    y_batches = []
    for i in range(num):
        X_batches.append(X[i*bs:(i+1)*bs])
        y_batches.append(y_train[i*bs:(i+1)*bs])

    return X_batches, y_batches


def cCrop(X, dim):
    p = transforms.Compose([
                        transforms.ToPILImage(),
                        transforms.CenterCrop(dim),
                        transforms.ToTensor()
    ])

    temp = torch.Tensor([])
    for i in X:
        a = p(i)
        temp = torch.cat([temp,a.unsqueeze(0)])
  
    return temp

In [7]:
X_train = cCrop(X_train,227)
X_train.shape

torch.Size([2669, 3, 227, 227])

In [ ]:
imsize = 227
# X_train = imageSetResize(imsize, X_train.float())
X_train = getNormalized(X_train.float(),imsize)
# X_test = imageSetResize(imsize, X_test.float())
# X_test = getNormalized(X_test.float(),imsize)

In [ ]:
class MyModel(nn.Module):
    def __init__(self):
        super(MyModel,self).__init__()
        self.conv1 = nn.Conv2d(3,96,kernel_size=11,stride = 4)
        self.pool1 = nn.MaxPool2d(3, stride = 2)
        self.bn1 = nn.BatchNorm2d(96)
        self.conv2 = nn.Conv2d(96,256,kernel_size=1,stride = 1)
        self.pool2 = nn.MaxPool2d(3, stride = 2)
        self.bn2 = nn.BatchNorm2d(256)
        self.conv3 = nn.Conv2d(256,384,kernel_size=1,stride = 1)
        self.conv4 = nn.Conv2d(384,384,kernel_size=1,stride = 1)
        self.conv5 = nn.Conv2d(384,256,kernel_size=1,stride = 1)
        self.pool3 = nn.MaxPool2d(3, stride = 2)
        self.ll1 = nn.Linear(9216,4096)
        self.ll2 = nn.Linear(4096,4096)
        self.ll3 = nn.Linear(4096,2)
        
        
            
    def forward(self,X):
        print('Image: {}'.format(X.shape))
        out = F.relu(self.conv1(X))
        print('conv1 + relu: {}'.format(out.shape))
        out = self.pool1(out)
        print('Pool1: {}'.format(out.shape))
        out = self.bn1(out)
        print('Batch Norm: {}'.format(out.shape))
        out = F.relu(self.conv2(out))
        print('conv2 + relu: {}'.format(out.shape))
        out = self.pool2(out)
        print('Pool2: {}'.format(out.shape))
        out = self.bn2(out)
        print('Batch Norm: {}'.format(out.shape))
        out = F.relu(self.conv3(out))
        print('conv3 + relu: {}'.format(out.shape))
        out = F.relu(self.conv4(out))
        print('conv4 + relu: {}'.format(out.shape))
        out = F.relu(self.conv5(out))
        print('conv5 + relu: {}'.format(out.shape))
        out = self.pool3(out)
        print('Pool3: {}'.format(out.shape))
        out = nn.Dropout(p=0.5)(out)
        print('Dropout: {}'.format(out.shape))
        out = F.relu(out.reshape(X.shape[0],-1))
        print('Flatten + relu: {}'.format(out.shape))
        out = F.relu(self.ll1(out))
        print('linear + relu: {}'.format(out.shape))
        out = F.relu(self.ll2(out))
        print('linear + relu: {}'.format(out.shape))
        out = F.relu(self.ll3(out))
        print('linear + relu: {}'.format(out.shape))
        sm = torch.softmax(out, dim=0)
        return sm

In [ ]:
m1 = MyModel()

In [ ]:
optimizer = optim.Adam(params = m1.parameters(),lr=1e-3) # Optimizer
# tb = SummaryWriter()
prev_testacc = -float('inf')


for epoch in range(100):
#     m1.train()
    print('Epoch {}'.format(epoch))
    X_batches, y_batches = shuffle_and_batch(X_train, y_train, 4, 67)
    for i in range(len(X_batches)):
        preds = m1(X_batches[i])
#         loss = criterion(preds,labelToOneHot(y_batches[i])) 
        loss = criterion(preds,y_batches[i].long())+ 0.65*sum(p.pow(2.0).sum() for p in m1.parameters())
        optimizer.zero_grad()
        loss.backward(retain_graph = True)
        optimizer.step()
    
    
#     m1.eval()
    # Checking model on training set
    train_preds = m1(X_train)
    train_loss = criterion(train_preds, y_train.long())
#     train_loss = criterion(train_preds,labelToOneHot(y_train))
    train_preds = labelize(train_preds)
    train_prediction_comparisons = (y_train == train_preds)
    train_accuracy = float(train_prediction_comparisons.sum())/float(y_train.shape[0])
    print('TrainLoss:{} TrainAccuracy:{}'.format(train_loss.item(), train_accuracy), end='  ')
    

    # Checking model on testing set
#     test_preds = m1(X_test)
#     test_loss = criterion(test_preds, y_test.long())
# #     test_loss = criterion(test_preds,labelToOneHot(y_test))
#     test_preds = labelize(test_preds)
#     test_prediction_comparisons = (y_test == test_preds)
#     test_accuracy = float(test_prediction_comparisons.sum())/float(y_test.shape[0])
#     print('TestLoss:{} TestAccuracy:{}'.format(test_loss.item(), test_accuracy))
#     if test_accuracy < prev_testacc and prev_testacc>0.7:
#         break
# #     state_dict = m1.state_dict()
# #     torch.save(state_dict, model_path)
#     prev_testacc = test_accuracy

    m1.eval()
    Xraw = pickle.load(open(f'X2669.pkl', 'rb'))
    yraw = pickle.load(open(f'y2669.pkl', 'rb'))

    raw_preds = m1(Xraw)
#     train_loss = criterion(raw_preds, yraw.long())
#     raw_loss = criterion(raw_preds, labelToOneHot(yraw)) #KLDivergence
    raw_preds = labelize(raw_preds)
    raw_prediction_comparisons = (yraw == raw_preds)
    raw_accuracy = float(raw_prediction_comparisons.sum())/float(yraw.shape[0])
#     print('RawAccuracy:{}'.format(raw_accuracy), end='\n') 
    
    
    cm = confusion_matrix(yraw, raw_preds)
    acc0 = cm[0][0]/cm[0].sum()
    acc1 = cm[1][1]/cm[1].sum()
    print('[0]{}% [1]{}%'.format(acc0*100.0, acc1*100.0))
    

In [ ]:
# aa = y_train == 0.
# aa[10:20]

In [ ]:
# a = X_train[10]
# b = cCrop(a.unsqueeze(0),227)

# plt.imshow(b[0].permute(1,2,0))

In [ ]:
# plt.imshow(X_train[10].permute(1,2,0))